In [ ]:
import nltk
import tqdm

In [ ]:
pip install spacy

In [ ]:
pip install '../input/raw-chineseconceptnet/zh_core_web_sm-3.1.0/dist/zh_core_web_sm-3.1.0.tar'

In [ ]:
conceptnet_path = '../input/raw-chineseconceptnet/chineseconceptnet.csv'
output_csv_path = '../input/raw-chineseconceptnet/conceptnet.zh.csv'
output_vocab_path = '../input/raw-chineseconceptnet/concept.txt'
output_path = 'matcher_patterns.zh.json'

In [ ]:
def create_pattern(nlp, doc, debug=False):
    pronoun_list = set(["我", "你", "它", "它的", "你的", "他", "她", "他的", "她的", "他们", "他们的", "我们的", "我们"])
    # Filtering concepts consisting of all stop words and longer than four words.
    if len(doc) >= 5 or doc[0].text in pronoun_list or doc[-1].text in pronoun_list:
        if debug:
            return False, doc.text
        return None  # ignore this concept as pattern

    pattern = []
    
    for token in doc:  # a doc is a concept
        pattern.append({"TEXT": token.text})
    if debug:
        return True, doc.text
    return pattern

In [ ]:
def load_cpnet_vocab(cpnet_vocab_path):
    with open(cpnet_vocab_path, "r", encoding="utf8") as fin:
        cpnet_vocab = [l.strip() for l in fin]
    cpnet_vocab = [c.replace("_", " ") for c in cpnet_vocab]
    return cpnet_vocab

In [ ]:
import spacy
import json
import pdb

In [ ]:
spacy.load('en_core_web_sm')

In [ ]:
def create_matcher_patterns(cpnet_vocab_path, output_path, debug=False):
    cpnet_vocab = load_cpnet_vocab(cpnet_vocab_path)
    nlp = spacy.load('zh_core_web_sm', disable=['parser', 'ner', 'textcat'])
    docs = nlp.pipe(cpnet_vocab)
    all_patterns = {}

    if debug:
        f = open("filtered_concept.txt", "w")

    for doc in docs:
        pattern = create_pattern(nlp, doc, debug)
        if debug:
            if not pattern[0]:
                f.write(pattern[1] + '\n')

        if pattern is None:
            continue
        all_patterns["_".join(doc.text.split(" "))] = pattern

    print("Created " + str(len(all_patterns)) + " patterns.")
    with open(output_path, "w", encoding="utf8") as fout:
        json.dump(all_patterns, fout, ensure_ascii=False)
    if debug:
        f.close()

create_matcher_patterns(output_vocab_path, output_path)